In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

# Add the parent directory to the system path
sys.path.append(str(Path.cwd().parent))

In [3]:
# Configure the log file path
LOG_FILE_PATH = Path("/Users/suyog/personal/sreAgent/output/logs.log")

In [4]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# Import custom tools
from tools.oncall_employees import GetOncallEmployeesTool
from tools.jira import CreateJiraTicketTool
from tools.file import FilteredLogReaderTool
from tools.stack_exchange import StackExchangeTool


def setup_agent():
    """Set up the ReAct agent with the necessary tools."""
    llm = ChatOpenAI(model="gpt-4o-mini")

    # Load ReAct prompt
    prompt = hub.pull("hwchase17/react")

    # Initialize tools
    tools = [
        FilteredLogReaderTool(log_file_path=LOG_FILE_PATH),
        # StackExchangeTool(),
        GetOncallEmployeesTool(),
        CreateJiraTicketTool(),
    ]

    # Create the agent
    agent = create_react_agent(llm, tools, prompt)

    # Create the agent executor
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=3,
    )

    return agent_executor


agent_executor = setup_agent()

ModuleNotFoundError: No module named 'base'

In [ ]:
prompt = (
    "Read logs from 2025-05-01 17:44:43 to 2025-05-01 17:44:44 and check for any errors or critical issues. "
    "Use the filtered_log_reader tool with these exact timestamps to only look at new logs since the last check."
    "You need to then idenify the potential cause and the possible solution for each of the error you saw."
    "After idenifying the potential cause and possible solution use get_oncall_employees tool to find filter out on-call employees best suited to handle each error"
    "Finally use create_jira_ticket to create appropriate tickets and assign it to the right employee"
)
response = agent_executor.invoke({"input": prompt})

In [ ]:
response